In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Insurance.csv")


In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[['charges']]

In [10]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': ['friedman_mse', 'mae','mse'],  # removed squared_error
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
  

grid = GridSearchCV(DecisionTreeRegressor(),
                    param_grid,
                    cv=5,
                    refit=True,
                    verbose=3,
                    n_jobs=-1)
grid.fit(x_train,y_train)


Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 626 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:    8.0s finished
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'mae', 'mse'],
                         'max_depth': [None, 5, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'm

In [14]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
print("The R_score value for best parameter{}: ".format(grid.best_params_))

The R_score value for best parameter{'criterion': 'mae', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'best'}: 


In [15]:
table=pd.DataFrame.from_dict(re)

In [16]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005687,0.003660,0.000309,0.000618,friedman_mse,None,1,2,best,"{'criterion': 'friedman_mse', 'max_depth': Non...",0.696508,0.483964,0.710236,0.623987,0.651874,0.633217,0.080897,263
1,0.003803,0.002062,0.000962,0.001925,friedman_mse,None,1,2,random,"{'criterion': 'friedman_mse', 'max_depth': Non...",0.647989,0.702841,0.638284,0.573331,0.676720,0.647895,0.043584,257
2,0.003395,0.001042,0.003493,0.002304,friedman_mse,None,1,5,best,"{'criterion': 'friedman_mse', 'max_depth': Non...",0.719526,0.542354,0.788827,0.658817,0.703146,0.682418,0.081667,242
3,0.003914,0.001671,0.001436,0.001459,friedman_mse,None,1,5,random,"{'criterion': 'friedman_mse', 'max_depth': Non...",0.670140,0.734987,0.773677,0.541456,0.660141,0.676139,0.079193,247
4,0.002940,0.002097,0.001909,0.001985,friedman_mse,None,1,10,best,"{'criterion': 'friedman_mse', 'max_depth': Non...",0.745904,0.613770,0.806387,0.678664,0.727970,0.714461,0.064895,235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0.003148,0.006296,0.003126,0.006251,mse,30,4,2,random,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.777876,0.733348,0.850727,0.772618,0.749211,0.776708,0.040312,104
266,0.006251,0.007655,0.000000,0.000000,mse,30,4,5,best,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.779705,0.715268,0.812573,0.766265,0.754971,0.765715,0.031800,153
267,0.009494,0.007770,0.000000,0.000000,mse,30,4,5,random,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.727672,0.786628,0.854807,0.789683,0.751761,0.782054,0.042996,85
268,0.004045,0.005878,0.003148,0.006296,mse,30,4,10,best,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.786245,0.709467,0.814027,0.775740,0.756606,0.768371,0.034854,129


In [17]:
print("✅ Best Score (CV):", grid.best_score_)

✅ Best Score (CV): 0.8298756267667614


In [22]:
y_pred = grid.best_estimator_.predict(x_test)
from sklearn.metrics import r2_score
r_score = r2_score(y_test, y_pred)
print("r² Score:", r_score)

r² Score: 0.8876266632751582


In [23]:
import pickle
filename="finalized_model_decisiontreeRegression.sav"

In [27]:
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [29]:
loaded_model=pickle.load(open("finalized_model_decisiontreeRegression.sav",'rb'))
result=loaded_model.predict([[19,27.900,0,0,1]])
result

array([47679.842675])